In [ ]:
%%time
%run -i 'nlp_implementation.py' #imports packages and inits alltext
from pyspark import SparkContext, SparkConf
import spacy
from spacy.tokens import DocBin

conf = SparkConf().setAppName("pyspark-shell").setMaster('local[*]').set("spark.executor.memory", "10g").set("spark.driver.memory", "10g").set('spark.driver.maxResultSize', "10G")
sc = SparkContext(conf=conf)
nlp = spacy.load("en_core_web_lg")
with open("cucumber/all_doc.bin", "rb") as w: #loading in all bytestreams of data
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
with open("cucumber/first_doc.bin", "rb") as w: #loading in
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 += list(new_docs1.get_docs(nlp.vocab))
print(len(docs1))

In [ ]:
%%time
import wikipedia
from googlesearch import search
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np

import os
import multiprocessing as mp
import subprocess
import concurrent.futures

def worker(name, vidId, dest):
    print("starting", name, vidId)
    ytid = "--youtubeid="+ vidId
    if not os.path.exists(dest.split("/")[0]):
        os.makedirs(dest.split("/")[0])

    while True:
        subprocess.run(["python3", "down.py", ytid, "--output", dest], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        if os.stat(dest).st_size != 0:
            print("done")
            break
        print("going again", name, vidId)

books_df = pd.read_csv('books_clean.csv')
def is_book1(name, df=books_df): #worker
#     print("trying", name)
#     db, wiki, goog, searches = False, False, False, False
    if name in df.title.values:
        return (name,True)
    try:
        summ = wikipedia.summary(name, sentences=3)
        if ent in summ:
            (name,True)
    except:
        pass


    links = search(name)
    for l in links:
        if bool(re.search("amazon.*dp", l)):
            return (name,True)
        if bool(re.search("books\.google.*"+name.replace(" ", "_").lower(), l.lower())):
            return (name,True)
        if bool(re.search("goodreads*"+name.replace(" ", "_").lower(), l.lower())):
            return (name,True)
        if bool(re.search("barnesandnoble*"+name.replace(" ", "_").lower(), l.lower())):
            return (name,True)
        if bool(re.search("penguinrandomhouse*"+name.replace(" ", "_").lower(), l.lower())):
            return (name,True)

    if sum([bool(re.search("book", l.lower())) for l in links])>4:
        return (name,True) 
    
    return (name, False)

        
def keep_stuff1(doc):
    books, people, places = [], [], []
    ents = doc.user_data["entis"]
    potential = ["NORP", "ORG", "PRODUCT", "WORK_OF_ART"]
    
    for ent, label in ents:
        if label=="PERSON": #save people entities
            people.append(ent)
        elif label=="LOC" or label=="GPE": #save location entites
            places.append(ent)
    #parallel processing to verify books
    with concurrent.futures.ThreadPoolExecutor(max_workers = 1) as executor:
        result = [executor.submit(is_book1, e[0]) for e in ents if e[1] in potential]
    for future in concurrent.futures.as_completed(result):
        if future.result()[1]==True:
            books.append(futures.result()[0])
    doc.user_data["books"] = books
    
    return doc
docs1 = [keep_stuff1(doc) for doc in docs1] #extract and save all entities by running above method

In [ ]:
%%time
from difflib import SequenceMatcher
all_peeps = []
for di in docs1:
    all_peeps +=di.user_data["people"]
#     all_peeps.append(di.user_data["guest"])
# print(len(all_peeps))
all_peeps = sorted(set(all_peeps))
# all_peeps = [(peo, 1) for peo in all_peeps]

i = 0

while i < len(all_peeps)-1:
    str1 = all_peeps[i]
#     print(i)
    str2 = all_peeps[i+1]
    
    if SequenceMatcher(a=str1,b=str2).ratio()>.8:
        all_peeps.remove(str1) if len(str1)>len(str2) else all_peeps.remove(str2)
    if str1 in str2:
        all_peeps.remove(str1)
    elif str2 in str1:
        all_peeps.remove(str2)
    i+=1

mentions_dict = dict(zip(all_peeps, np.arange(len(all_peeps))))
speakers = [guest.user_data["guest"] for guest in docs1]
speaker_dict = dict(zip(speakers, np.arange(len(all_peeps), len(all_peeps)+len(speakers))))

one_speaker_mention = []
for doc in docs1:
    cur_id = speaker_dict.get(doc.user_data["guest"])
    for ment in doc.user_data["people"]:
        if mentions_dict.get(ment):
            one_speaker_mention.append((cur_id, mentions_dict.get(ment), 1))

i = len(all_peeps)+len(speakers)

people_edges_dict = dict(zip(np.arange(i, i+len(one_speaker_mention)), one_speaker_mention))

import csv
with open('peopleNodes.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "name", "group"])
    for key, value in mentions_dict.items():
        writer.writerow([value, key, 2])
    for key, value in speaker_dict.items():
        writer.writerow([value, key, 1])
with open('peopleEdges.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "source", "target", "value"])
    for key, value in people_edges_dict.items():
        writer.writerow([key, value[0], value[1], value[2]])

In [ ]:
%%time #next three cells manipulate saved entities and create dictionaries to csv
all_places = []
for di in docs1:
    all_places +=di.user_data["places"]
all_places = sorted(set(all_places))
# all_peeps = [(peo, 1) for peo in all_peeps]

i = 0

while i < len(all_places)-1:
    str1 = all_places[i]
#     print(i)
    str2 = all_places[i+1]
    
    if SequenceMatcher(a=str1,b=str2).ratio()>.8:
        all_places.remove(str1) if len(str1)>len(str2) else all_places.remove(str2)
    if str1 in str2:
        all_places.remove(str1)
    elif str2 in str1:
        all_places.remove(str2)
    i+=1

visited_dict = dict(zip(all_places, np.arange(len(all_places))))
places = [guest.user_data["guest"] for guest in docs1]
place_dict = dict(zip(places, np.arange(len(all_places), len(all_places)+len(places))))

one_place_mention = []
for doc in docs1:
    cur_id = place_dict.get(doc.user_data["guest"])
    for ment in doc.user_data["people"]:
        if place_dict.get(ment):
            one_place_mention.append((cur_id, mentions_dict.get(ment), 1))

i = len(all_places)+len(places)

place_edges_dict = dict(zip(np.arange(i, i+len(one_place_mention)), one_place_mention))

import csv
with open('placeNodes.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "name", "group"])
    for key, value in place_dict.items():
        writer.writerow([value, key, 2])
    for key, value in visited_dict.items():
        writer.writerow([value, key, 1])
with open('placeEdges.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "source", "target", "value"])
    for key, value in place_edges_dict.items():
        writer.writerow([key, value[0], value[1], value[2]])

In [ ]:
%%time
all_books = []
for di in docs1:
    all_books +=di.user_data["books"]
all_books = sorted(set(all_books))
i = 0

while i < len(all_books)-1:
    str1 = all_books[i]
#     print(i)
    str2 = all_books[i+1]
    
    if SequenceMatcher(a=str1,b=str2).ratio()>.8:
        all_books.remove(str1) if len(str1)>len(str2) else all_books.remove(str2)
    if str1 in str2:
        all_books.remove(str1)
    elif str2 in str1:
        all_books.remove(str2)
    i+=1

read_book = dict(zip(all_books, np.arange(len(all_books))))
books = [guest.user_data["guest"] for guest in docs1]
book_dict = dict(zip(books, np.arange(len(all_books), len(all_books)+len(books))))

one_book_mention = []
for doc in docs1:
    cur_id = book_dict.get(doc.user_data["guest"])
    for ment in doc.user_data["people"]:
        if book_dict.get(ment):
            one_book_mention.append((cur_id, book_dict.get(ment), 1))

i = len(all_places)+len(places)

book_edges_dict = dict(zip(np.arange(i, i+len(one_book_mention)), one_book_mention))

import csv
with open('bookNodes.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "name", "group"])
    for key, value in book_dict.items():
        writer.writerow([value, key, 2])
    for key, value in read_book.items():
        writer.writerow([value, key, 1])
with open('bookEdges.csv', 'w+') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id", "source", "target", "value"])
    for key, value in book_edges_dict.items():
        writer.writerow([key, value[0], value[1], value[2]])